<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Чем отличаются современные сверточные сети от сетей 5 летней давности?

Они стали очень сильно глубже

**Вопрос 2**: Какие неприятности могут возникнуть во время обучения современных нейросетей?

Если они слишком глубокие, то может затухать градиент - для этого делают как бы подсети, которые обучаются отдельно (то есть по пути есть какой-то output, который позволяет протолкнуть ошибку чтобы она не затухла к инпуту)

**Вопрос 3**: У вас есть очень маленький датасет из 100 картинок, классификация, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить? что делать если первый вариант  решения не заработает?

переобучение (в каждом классе мало элементов, значит будут запоминаться слишком специфические признаки) - можно попробовать добавлять шум на картинки или попытаться как-то их нарезать.
- в классе собачек они все на фоне жёлтой стены? вырежем собачек. 
- в классе котят одни шотландские? пошумим в цветовых каналах или обесцветим
- зеркально отразим

Если ничего не получится, пойду на толоку и заставлю больше размечать картинки

**Вопрос 4**: Как сделать стайл трансфер для музыки? oO

Так же, как это делается с картинками. Важно придумать представление для музыки - можно закодировать её в узкую картинку:
есть 10^6 частот, возьмём картинку высотой 10^6, в каждом столбике - распределение частот в этот момент времени. Дальше сделаем какой-нибудь антиалиасинг или по-другому пожмём картинку (например, кластеризуем кнн-ом и оставим жирные центры кластеров) - получим опять сильно вытянутую по горизонтали картинку, но уже с адекватной размерностью - можно фигачить перенос стилей:
для донора нарежем эту картинку на квадратики (с той же высотой, может быть перекрывающиеся) и сделаем отдельный перенос для каждого квадрата (мб прямоугольника, если жалко ресурсов). будем переносить стили с квадрата-донора на квадрат-получателя (если их не равное колво, можно же прыгать, главное чтобы его номер был первым в том же 0.1-квантиле например)

In [18]:
import sys
import os
import time
import string
import random
import pickle
import cPickle

In [19]:
import numpy as np
import theano
import theano.tensor as T
import lasagne

In [20]:
from lasagne import init
from lasagne.layers import *
from lasagne.nonlinearities import softmax, rectify
import lasagne.nonlinearities

In [21]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def iterate_minibatches(inputs, targets, batchsize, shuffle=False, augment=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        if augment:
            # as in paper : 
            # pad feature arrays with 4 pixels on each side
            # and do random cropping of 32x32
            padded = np.pad(inputs[excerpt],((0,0),(0,0),(4,4),(4,4)),mode='constant')
            random_cropped = np.zeros(inputs[excerpt].shape, dtype=np.float32)
            crops = np.random.random_integers(0,high=8,size=(batchsize,2))
            for r in range(batchsize):
                random_cropped[r,:,:,:] = padded[r,:,crops[r,0]:(crops[r,0]+32),crops[r,1]:(crops[r,1]+32)]
            inp_exc = random_cropped
        else:
            inp_exc = inputs[excerpt]

        yield inp_exc, targets[excerpt]

In [22]:
n=5
num_epochs=82

xs = []
ys = []
for j in range(5):
    d = unpickle('cifar-10-batches-py/data_batch_'+'j+1')
    x = d['data']
    y = d['labels']
    xs.append(x)
    ys.append(y)

d = unpickle('cifar-10-batches-py/test_batch')
xs.append(d['data'])
ys.append(d['labels'])

x = np.concatenate(xs)/np.float32(255)
y = np.concatenate(ys)
x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
x = x.reshape((x.shape[0], 32, 32, 3)).transpose(0,3,1,2)

pixel_mean = np.mean(x[0:50000],axis=0)
x -= pixel_mean

In [23]:
X_train = x[0:50000,:,:,:]
Y_train = y[0:50000]
X_train_flip = X_train[:,:,:,::-1]
Y_train_flip = Y_train
X_train = np.concatenate((X_train,X_train_flip),axis=0)
Y_train = np.concatenate((Y_train,Y_train_flip),axis=0)

X_test = x[50000:,:,:,:]
Y_test = y[50000:]

data = dict(
        X_train=lasagne.utils.floatX(X_train),
        Y_train=Y_train.astype('int32'),
        X_test = lasagne.utils.floatX(X_test),
        Y_test = Y_test.astype('int32'),)

X_train = data['X_train']
Y_train = data['Y_train']
X_test = data['X_test']
Y_test = data['Y_test']

input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

In [24]:
def residual_block(l, increase_dim=False, projection=False):
    input_num_filters = l.output_shape[1]
    if increase_dim:
        first_stride = (2,2)
        out_num_filters = input_num_filters*2
    else:
        first_stride = (1,1)
        out_num_filters = input_num_filters

    stack_1 = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(3,3), stride=first_stride, 
                                   nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), 
                                   flip_filters=False))
    stack_2 = batch_norm(ConvLayer(stack_1, num_filters=out_num_filters, filter_size=(3,3), stride=(1,1), 
                                   nonlinearity=None, pad='same', W=lasagne.init.HeNormal(gain='relu'), 
                                   flip_filters=False))

    # add shortcut connections
    if increase_dim:
        if projection:
            # projection shortcut, as option B in paper
            projection = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(1,1), 
                                              stride=(2,2), nonlinearity=None, pad='same', b=None, 
                                              flip_filters=False))
            block = NonlinearityLayer(ElemwiseSumLayer([stack_2, projection]),nonlinearity=rectify)
        else:
            # identity shortcut, as option A in paper
            identity = ExpressionLayer(l, lambda X: X[:, :, ::2, ::2], lambda s: (s[0], s[1], s[2]//2, s[3]//2))
            padding = PadLayer(identity, [out_num_filters//4,0,0], batch_ndim=1)
            block = NonlinearityLayer(ElemwiseSumLayer([stack_2, padding]),nonlinearity=rectify)
    else:
        block = NonlinearityLayer(ElemwiseSumLayer([stack_2, l]),nonlinearity=rectify)

    return block

In [25]:
l_in = InputLayer(shape=(None, 3, 32, 32), input_var=input_var)
l = batch_norm(ConvLayer(l_in, num_filters=16, 
                         filter_size=(3,3), stride=(1,1), 
                         nonlinearity=rectify, pad='same', 
                         W=lasagne.init.HeNormal(gain='relu'), 
                         flip_filters=False))

In [26]:
for _ in range(n):
    l = residual_block(l)
l = residual_block(l, increase_dim=True)
for _ in range(1,n):
    l = residual_block(l)
l = residual_block(l, increase_dim=True)
for _ in range(1,n):
    l = residual_block(l)
l = GlobalPoolLayer(l)
network = DenseLayer(
        l, num_units=10,
        W=lasagne.init.HeNormal(),
        nonlinearity=softmax)

In [27]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()
all_layers = lasagne.layers.get_all_layers(network)
l2_penalty = lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2) * 0.0001
loss = loss + l2_penalty

In [28]:
params = lasagne.layers.get_all_params(network, trainable=True)
lr = 0.1
sh_lr = theano.shared(lasagne.utils.floatX(lr))
updates = lasagne.updates.momentum(
        loss, params, learning_rate=sh_lr, momentum=0.9)

In [29]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [30]:
for epoch in range(num_epochs):
    train_indices = np.arange(100000)
    np.random.shuffle(train_indices)
    X_train = X_train[train_indices,:,:,:]
    Y_train = Y_train[train_indices]

    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, Y_train, 128, shuffle=True, augment=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, Y_test, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    if (epoch+1) == 41 or (epoch+1) == 61:
        new_lr = sh_lr.get_value() * 0.1
        sh_lr.set_value(lasagne.utils.floatX(new_lr))

In [31]:
np.savez('cifar10_deep_residual_model.npz', *lasagne.layers.get_all_param_values(network))

In [32]:
#     with np.load(model) as f:
#          param_values = [f['arr_%d' % i] for i in range(len(f.files))]
#     lasagne.layers.set_all_param_values(network, param_values)

In [33]:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, Y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1

In [34]:
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

Final results:
  test loss:			0.327143
  test accuracy:		92.65 %


In [35]:
!pwd

/home/alaktionov/alaktionov_oneshots/notebooks


In [ ]:
# !mkdir cifar10
# !sudo curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
# !tar -xvzf cifar-10-python.tar.gz -C cifar10

In [ ]:
# import matplotlib
# matplotlib.use('Agg')

# import os
# import numpy as np
# import pandas as pd

# %matplotlib inline

In [ ]:
# from sklearn.cross_validation import StratifiedKFold
# from sklearn.preprocessing import LabelEncoder


In [ ]:
# from cifar import load_CIFAR10
# plt.rcParams['figure.figsize'] = (10.0, 8.0) 

# cifar10_dir = './cifar10/cifar-10-batches-py'
# X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [ ]:

def stratified_train_test_split(X, y, test_size=0.25, random_state=None):
    n_folds = int(1 / test_size)
    skf = StratifiedKFold(y, n_folds=n_folds, random_state=random_state)
    train_idx, test_idx = iter(skf).next()
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]


def load_data(test_size=0.25, random_state=None, 
              data_dir='/home/alaktionov/alaktionov_oneshots/notebooks/examples/cifar10/data'):
    csv_fname = os.path.join(data_dir, 'trainLabels.csv')
    df = pd.read_csv(csv_fname)
    X = df['id'].apply(lambda i: '%s.png' % i).values
    y = LabelEncoder().fit_transform(df['label'].values)

    X_train, X_test, y_train, y_test = stratified_train_test_split(X, y, test_size=test_size, random_state=random_state)
    y_train, y_test = y_train.astype(np.int32), y_test.astype(np.int32)
    return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = load_data(test_size=0.2, random_state=42)

In [ ]:
from nolearn.lasagne.handlers import SaveWeights


from nolearn_utils.iterators import (
    BufferedBatchIteratorMixin,
    ReadImageBatchIteratorMixin,
    ShuffleBatchIteratorMixin,
    RandomFlipBatchIteratorMixin,
    AffineTransformBatchIteratorMixin,
    make_iterator
)
from nolearn_utils.hooks import SaveTrainingHistory, PlotTrainingHistory

from nolearn_utils.hooks import SaveTrainingHistory, PlotTrainingHistory

batch_size = 32
n_classes = 10
image_size = 32

train_iterator_mixins = [
    ShuffleBatchIteratorMixin,
    ReadImageBatchIteratorMixin,
    RandomFlipBatchIteratorMixin,
    AffineTransformBatchIteratorMixin,
    BufferedBatchIteratorMixin,
]
TrainIterator = make_iterator('TrainIterator', train_iterator_mixins)

test_iterator_mixins = [
    ReadImageBatchIteratorMixin,
    BufferedBatchIteratorMixin,
]
TestIterator = make_iterator('TestIterator', test_iterator_mixins)

train_iterator_kwargs = {
    'buffer_size': 5,
    'batch_size': batch_size,
    'read_image_size': (image_size, image_size),
    'read_image_as_gray': False,
    'read_image_prefix_path': '/home/alaktionov/alaktionov_oneshots/notebooks/examples/cifar10/data/train/',
    'flip_horizontal_p': 0.5,
    'flip_vertical_p': 0,
    'affine_p': 0.5,
    'affine_scale_choices': np.linspace(0.75, 1.25, 5),
    'affine_shear_choices': np.linspace(0.75, 1.25, 5),
    'affine_translation_choices': np.arange(-3, 4, 1),
    'affine_rotation_choices': np.arange(-45, 50, 5)
}
train_iterator = TrainIterator(**train_iterator_kwargs)

test_iterator_kwargs = {
    'buffer_size': 5,
    'batch_size': batch_size,
    'read_image_size': (image_size, image_size),
    'read_image_as_gray': False,
    'read_image_prefix_path': '/home/alaktionov/alaktionov_oneshots/notebooks/examples/cifar10/data/train/'
}
test_iterator = TestIterator(**test_iterator_kwargs)

save_weights = SaveWeights('/home/alaktionov/alaktionov_oneshots/notebooks/examples/cifar10/model_weights.pkl', only_best=True, pickle=False)
save_training_history = SaveTrainingHistory('/home/alaktionov/alaktionov_oneshots/notebooks/examples/cifar10/model_history.pkl')
plot_training_history = PlotTrainingHistory('/home/alaktionov/alaktionov_oneshots/notebooks/examples/cifar10/training_history.png')


In [ ]:
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# num_classes = len(classes)
# samples_per_class = 7
# for y, cls in enumerate(classes):
#     idxs = np.flatnonzero(y_train == y)
#     idxs = np.random.choice(idxs, samples_per_class, replace=False)
#     for i, idx in enumerate(idxs):
#         plt_idx = i * num_classes + y + 1
#         plt.subplot(samples_per_class, num_classes, plt_idx)
#         plt.imshow(X_train[idx].astype('uint8').transpose(1, 2, 0))
#         plt.axis('off')
#         if i == 0:
#             plt.title(cls)
# plt.show()

In [ ]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

# input_X = T.tensor4("X")
# target_y = T.vector("target Y integer",dtype='int32')


In [ ]:
# net = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_X)

# # # net = <сверочная нейросеть>

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=32, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=32, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=32, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=48, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=48, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.MaxPool2DLayer(net, 
#                                     pool_size=(2,2))

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=80, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=80, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=80, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=80, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=80, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.MaxPool2DLayer(net, 
#                                     pool_size=(2,2))

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=128, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=128, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=128, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=128, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# net = lasagne.layers.Conv2DLayer(net, 
#                                  num_filters=128, 
#                                  filter_size=(3,3), 
#                                  stride=(1,1),
#                                  pad=(2,2),
#                                  W=init.Orthogonal(2**.5),
#                                  nonlinearity=lasagne.nonlinearities.rectify)

# # net = lasagne.layers.GlobalPoolLayer(net,
# #                                      pool_size=(8,8))

# net = lasagne.layers.Pool2DLayer(net, 
#                                  pool_size=(8,8))


# net = lasagne.layers.DenseLayer(net,
#                                 num_units=500)

# net = lasagne.layers.DenseLayer(net,
#                                 num_units=10,
#                                 nonlinearity=softmax)



In [ ]:
# from lasagne.layers.dnn import Conv2DDNNLayer, MaxPool2DDNNLayer
from nolearn.lasagne import NeuralNet
from lasagne.layers import Conv2DLayer, MaxPool2DLayer, FeaturePoolLayer, DropoutLayer

In [ ]:
net = NeuralNet(
    layers=[
        (InputLayer, dict(name='in', shape=(None, 3, image_size, image_size))),

        (Conv2DLayer, dict(name='l1c1', num_filters=32, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c2', num_filters=32, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c3', num_filters=32, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c4', num_filters=48, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c5', num_filters=48, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (MaxPool2DLayer, dict(name='l1p', pool_size=(2,2), stride=(1,1))),

        (Conv2DLayer, dict(name='l2c1', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c2', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c3', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c4', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c5', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (MaxPool2DLayer, dict(name='l2p', pool_size=(2,2), stride=(1,1))),
        
        (Conv2DLayer, dict(name='l3c1', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c2', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c3', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c4', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c5', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (MaxPool2DLayer, dict(name='l3p', pool_size=(8,8), stride=(1,1))),

        (DenseLayer, dict(name='l4', num_units=500)),

        (DenseLayer, dict(name='out', num_units=n_classes, nonlinearity=lasagne.nonlinearities.softmax)),
    ],
    
    regression=False,
    objective_loss_function=lasagne.objectives.categorical_crossentropy,

    update=lasagne.updates.adam,

    batch_iterator_train=train_iterator,
    batch_iterator_test=test_iterator,

    on_epoch_finished=[
        save_weights,
        save_training_history,
        plot_training_history
    ],

    verbose=10,
    max_epochs=250,
)

In [ ]:
# y_predicted = lasagne.layers.get_output(net)
# all_weights = lasagne.layers.get_all_params(net)
# print all_weights

In [ ]:
# prediction = lasagne.layers.get_output(net)
# loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()

# updates = lasagne.updates.adam(loss, all_weights)#, learning_rate=sh_lr)
# accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [ ]:
# train_fun = theano.function([input_X, target_y], [loss, accuracy], updates=updates, allow_input_downcast=True)
# accuracy_fun = theano.function([input_X, target_y], accuracy, allow_input_downcast=True)

### Вот и всё, пошли её учить

In [ ]:
# def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
#     assert len(inputs) == len(targets)
#     if shuffle:
#         indices = np.arange(len(inputs))
#         np.random.shuffle(indices)
#     for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
#         if shuffle:
#             excerpt = indices[start_idx:start_idx + batchsize]
#         else:
#             excerpt = slice(start_idx, start_idx + batchsize)
#         yield inputs[excerpt], targets[excerpt]

# Процесс обучения

In [ ]:
# import time
# import sys

# num_epochs = 100 #количество проходов по данным

# batch_size = 50 #размер мини-батча

# for epoch in range(num_epochs):
#     # In each epoch, we do a full pass over the training data:
#     train_err = 0
#     train_acc = 0
#     train_batches = 0
#     start_time = time.time()
#     for batch in iterate_minibatches(X_train, y_train,batch_size):
#         inputs, targets = batch
#         train_err_batch, train_acc_batch = train_fun(inputs, targets)
#         train_err += train_err_batch
#         train_acc += train_acc_batch
#         train_batches += 1

#     # And a full pass over the validation data:
#     print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
#     print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
#     print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
        
#     val_acc = 0
#     val_batches = 0
#     for batch in iterate_minibatches(X_test, y_test, batch_size):
#         inputs, targets = batch
#         val_acc += accuracy_fun(inputs, targets)
#         val_batches += 1

#     # Then we print the results for this epoch:

#     print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))
    
#     sys.stdout.flush()


In [ ]:
# test_acc = 0
# test_batches = 0
# for batch in iterate_minibatches(X_test, y_test, 500):
#     inputs, targets = batch
#     acc = accuracy_fun(inputs, targets)
#     test_acc += acc
#     test_batches += 1
# print("Final results:")
# print("  test accuracy:\t\t{:.2f} %".format(
#     test_acc / test_batches * 100))

# if test_acc / test_batches * 100 > 2.5:
#     print "Achievement unlocked: колдун 80 уровня"
# else:
#     print "Нужно больше магии!"

In [ ]:
net.fit(X_train, y_train)

In [ ]:
# net.score(X_test, y_test)

## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 